In [3]:
p=2^372*3^239-1 
K.<a>=GF(p^2,'a',modulus=[1,0,1])
root=a
K=GF(p^2,'a',modulus=[1,0,1])
E=EllipticCurve(K,[1,0])
n=372
C=E(0,0)
Infi=(2*C)
def distorsion_map(P):
    x=-P[0]
    y=P[1]*root
    return E(x,y)
# generates 3 torsion
P=E(6,4688558410880967105802548485745508960029362498156488028653160515710832445592615410221556012727376816077230428365690034126899246042233876329216033729810811325868800516984641936994142700675815111949425502583260986657458474705646)
P_A=(2^372)*P
Q_A=distorsion_map(P_A)
# generates 2 torsion
P2=E(11,4178137817326599126129602268030084854254919541457752920496919677484559856346888397586423437889634773403442208976556528418523132894243487824255334881927098101022506929946230877638251483704236124248972585701878366089218056349632)
P_B=(3^239)*P2
Q_B=distorsion_map(P_B)
def three_point_ladder(P,Q,alpha):
    if(alpha==0):
        return P
    else:
        P_Q=(P-Q)[0]
        digit_store=[]
        while(alpha):
            digit_store.append(alpha&1)
            alpha=alpha//2
        digit_store.reverse()
        flag=0
        A_X=0
        A_Z=0
        B_X=Q[0]
        B_Z=1
        C_X=P[0]
        C_Z=1
        while (len(digit_store)!=0):
            c=digit_store[0]
            temp_A_X=A_X
            temp_A_Z=A_Z
            temp_B_X=B_X
            temp_B_Z=B_Z
            temp_C_X=C_X
            temp_C_Z=C_Z
            if(c&1==1 and flag==0):
                A_X=Q[0]
                A_Z=1
                # DOUBLE B
                B_X,B_Z=(temp_B_X^2-temp_B_Z^2)^2,4*temp_B_X*temp_B_Z*(temp_B_X^2+temp_B_Z^2)
                # ladder on C=dadd(B,C,Q-P)
                C_X,C_Z=(temp_B_X*temp_C_X-temp_B_Z*temp_C_Z)^2,P_Q*(temp_B_X*temp_C_Z-temp_B_Z*temp_C_X)^2
                flag=1
            elif(c&1==0 and flag==1):
                # DOUBLE A
                A_X,A_Z=(temp_A_X^2-temp_A_Z^2)^2,4*temp_A_X*temp_A_Z*(temp_A_X^2+temp_A_Z^2)
                # ladder on B=dadd(A,B,Q)
                B_X,B_Z=(temp_A_X*temp_B_X-temp_A_Z*temp_B_Z)^2,Q[0]*(temp_A_X*temp_B_Z-temp_A_Z*temp_B_X)^2
                # ladder on C=dadd(A,C,P)
                C_X,C_Z=(temp_A_X*temp_C_X-temp_A_Z*temp_C_Z)^2,P[0]*(temp_A_X*temp_C_Z-temp_A_Z*temp_C_X)^2
            elif(c&1==1 and flag==1):
                # ladder on A=dadd(A,B,Q)
                A_X,A_Z=(temp_A_X*temp_B_X-temp_A_Z*temp_B_Z)^2,Q[0]*(temp_A_X*temp_B_Z-temp_A_Z*temp_B_X)^2
                # DOUBLE B
                B_X,B_Z=(temp_B_X^2-temp_B_Z^2)^2,4*temp_B_X*temp_B_Z*(temp_B_X^2+temp_B_Z^2)
                # ladder on C=dadd(B,C,Q-P)
                C_X,C_Z=(temp_B_X*temp_C_X-temp_B_Z*temp_C_Z)^2,P_Q*((temp_B_X*temp_C_Z)-(temp_B_Z*temp_C_X))^2
            elif(c&1==0 and flag==0):
                continue
            digit_store.pop(0)
        return((C_X*(C_Z^-1)),(A_X*(A_Z^-1)))
def okeya_sakurai(P,Q,alpha):
    x=P[0]
    y=P[1]
    result=three_point_ladder(P,Q,alpha)
    x2=result[0]
    x1=result[1]
    y2=((x+x2)*((x*x2)+1)-((x2-x)^2)*x1)*((2*y)^-1)
    return E(x2,-y2)
def isogeny_builder_three(R,E):
    R_out=R
    E_out=E
    phi_chain=[]
        stor=3^(238)
    for i in range(0,239):
        temp=stor*R_out
        phi=E_out.isogeny(temp)
        E_out=phi.codomain()
        phi_chain.append(phi)
        R_out=phi(R_out)
        stor=stor//3
    return (E_out,R_out,phi_chain)
def secret_isogeny(R,E,Q):
    phi_list=isogeny_builder_three(R,E)[2]
    P=Q
    for i in range(0,len(phi_list)):
        P=phi_list[i](P)
    return P
%timeit(secret_isogeny(okeya_sakurai(P_A,Q_A,3),E,P_B))
print(secret_isogeny(okeya_sakurai(P_A,Q_A,3),E,P_B))

1 loop, best of 3: 36.1 s per loop
(1427944523403092169485450975100487536384424150518011390020805345360109795012866939649544843728717630141849598878326816508202108166628768847551740081218126862562303056042470004365904499469226053177380676259026506255769234968540 : 4521016351757162037569232779246133415842595782643397244955666464351460248749533032779488202935582064223775123229263179181083961223831446329080278822898538635745059081128438420237787244976397483611885400805593528149506134164430 : 1)
